In [1]:
import numpy as np
import pandas as pd
import os
import math
from operator import itemgetter
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KDTree
from sklearn.model_selection import ParameterGrid

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from tqdm import tqdm

In [2]:
from mymodule.cluster2 import Cluster
from mymodule.track import Track, TrackPool
from mymodule.merger import Merge_1, Merge_2
from mymodule.utility import Score, CreateSubmission, Extend

In [3]:
# load data
event_id=1000
parameter = {}
parameter['event_id'] = 1000
parameter['is_training'] = False

path_to_train = '/home/yang/Study/kaggle/trackml/data/train_sample/train_100_events'
hits, cells, particles, truth = load_event(os.path.join(path_to_train, 'event00000'+str(parameter['event_id'])))

if parameter['is_training']:
    data = pd.merge(hits, truth, on='hit_id', how='left')
else:
    data = hits

score = []

print(len(data))
data.head()

120939


,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-64.409897,-7.163700,-1502.5,7,2,1
1,2,-55.336102,0.635342,-1502.5,7,2,1
2,3,-83.830498,-1.143010,-1502.5,7,2,1
3,4,-96.109100,-8.241030,-1502.5,7,2,1
4,5,-62.673599,-9.371200,-1502.5,7,2,1


In [4]:
%%time
# track with length>=14
# cluster

parameter={}

# rotate on r parameters
parameter['use_rotate_on_r'] = True
parameter['rotate_on_r_w1'] = 0.4
parameter['rotate_on_r_w2'] = 0.2
parameter['rotate_on_r_w3'] = 0.1
parameter['rotate_on_r_quad_coef'] = 0.000006
parameter['rotate_on_r_max_iter'] = 200
parameter['rotate_on_r_eps'] = 0.0035
parameter['rotate_on_r_step_eps'] = 0.0000

# rotate on theta parameters
parameter['use_rotate_on_theta'] = False
parameter['rotate_on_theta_eps'] = 0.0028
parameter['rotate_on_theta_max_iter'] = 720

# shift on z parameters
parameter['use_shift_on_z'] = True

# multiprocess
parameter['use_multiprocess'] = True

####
clusterer = Cluster(data, parameter)
result = clusterer.run()
####

# create track pool

parameter={}
parameter['track_length_1_min']=14
parameter['track_length_1_max']=20
parameter['use_multiprocess'] = True
parameter['is_training'] = False

####
pool = TrackPool(data, result, parameter)
print(pool.length_1)
print(pool.length_2)
####

# merge_1
parameter={}
parameter['is_training'] = False
parameter['track_length_2_min'] = 14
parameter['track_length_2_max'] = 20
parameter['track_length_diff_max'] = 4

####
output = Merge_1(pool, parameter)
print(len(output))
data = data.merge(output, how='left', on='hit_id')
####
data.head()

# score
print(Score(event_id, data, truth))
print(len(data.loc[data.track_id.notnull()]))
print(len(data.loc[data.track_id.notnull()])/len(data))
score.append((parameter['track_length_2_min'], parameter['track_length_diff_max'], Score(event_id, data, truth)))

100%|██████████| 200/200 [05:46<00:00,  1.73s/it]


68960
4737
11008
0.09350334346239389
11008
0.09102109327842962
CPU times: user 1min 7s, sys: 3.83 s, total: 1min 11s
Wall time: 7min 38s


In [5]:
%%time
# track with length>=12
# cluster

parameter={}

# rotate on r parameters
parameter['use_rotate_on_r'] = True
parameter['rotate_on_r_w1'] = 0.4
parameter['rotate_on_r_w2'] = 0.2
parameter['rotate_on_r_w3'] = 0.1
parameter['rotate_on_r_quad_coef'] = 0.000006
parameter['rotate_on_r_max_iter'] = 200
parameter['rotate_on_r_eps'] = 0.0035
parameter['rotate_on_r_step_eps'] = 0.0000

# rotate on theta parameters
parameter['use_rotate_on_theta'] = False
parameter['rotate_on_theta_eps'] = 0.0028
parameter['rotate_on_theta_max_iter'] = 720

# shift on z parameters
parameter['use_shift_on_z'] = True

# multiprocess
parameter['use_multiprocess'] = True

####
clusterer = Cluster(data.loc[data.track_id.isnull()], parameter)
result = clusterer.run()
####

# create track pool

parameter={}
parameter['track_length_1_min']=12
parameter['track_length_1_max']=20
parameter['use_multiprocess'] = True
parameter['is_training'] = False

####
pool = TrackPool(data.loc[data.track_id.isnull()], result, parameter)
print(pool.length_1)
print(pool.length_2)
####

# merge_1
parameter={}
parameter['is_training'] = False
parameter['track_length_2_min'] = 12
parameter['track_length_2_max'] = 20
parameter['track_length_diff_max'] = 4

####
output = Merge_1(pool, parameter)
print(len(output))
temp = data.merge(output, how='left', on='hit_id', suffixes=('_old', ''))
data.update(temp.track_id)
####
data.head()

# score
print(Score(event_id, data, truth))
print(len(data.loc[data.track_id.notnull()]))
print(len(data.loc[data.track_id.notnull()])/len(data))
score.append((parameter['track_length_2_min'], parameter['track_length_diff_max'], Score(event_id, data, truth)))

100%|██████████| 200/200 [04:58<00:00,  1.49s/it]


97546
7510
20464
0.2855895408751131
31472
0.26023036406783584
CPU times: user 2min, sys: 7.56 s, total: 2min 7s
Wall time: 7min 58s


In [6]:
%%time
# track with length>=12
# cluster

parameter={}

# rotate on r parameters
parameter['use_rotate_on_r'] = False
parameter['rotate_on_r_w1'] = 0.4
parameter['rotate_on_r_w2'] = 0.2
parameter['rotate_on_r_w3'] = 0.1
parameter['rotate_on_r_quad_coef'] = 0.000006
parameter['rotate_on_r_max_iter'] = 200
parameter['rotate_on_r_eps'] = 0.0035
parameter['rotate_on_r_step_eps'] = 0.0000

# rotate on theta parameters
parameter['use_rotate_on_theta'] = True
parameter['rotate_on_theta_eps'] = 0.0027
parameter['rotate_on_theta_max_iter'] = 1080

# shift on z parameters
parameter['use_shift_on_z'] = True

# multiprocess
parameter['use_multiprocess'] = True

####
clusterer = Cluster(data.loc[data.track_id.isnull()], parameter)
result = clusterer.run()
####

# create track pool

parameter={}
parameter['track_length_1_min']=12
parameter['track_length_1_max']=20
parameter['use_multiprocess'] = True
parameter['is_training'] = False

####
pool = TrackPool(data.loc[data.track_id.isnull()], result, parameter)
print(pool.length_1)
print(pool.length_2)
####

# merge_1
parameter={}
parameter['is_training'] = False
parameter['track_length_2_min'] = 12
parameter['track_length_2_max'] = 20
parameter['track_length_diff_max'] = 4

####
output = Merge_1(pool, parameter)
print(len(output))
temp = data.merge(output, how='left', on='hit_id', suffixes=('_old', ''))
data.update(temp.track_id)
####
data.head()

# score
print(Score(event_id, data, truth))
print(len(data.loc[data.track_id.notnull()]))
print(len(data.loc[data.track_id.notnull()])/len(data))
score.append((parameter['track_length_2_min'], parameter['track_length_diff_max'], Score(event_id, data, truth)))

100%|██████████| 1080/1080 [12:27<00:00,  1.44it/s]


10341
4856
6487
0.32533472275248315
37959
0.3138689752685238
CPU times: user 52.5 s, sys: 47.4 s, total: 1min 39s
Wall time: 14min 8s


In [7]:
%%time
# track with length>=10
# cluster

parameter={}

# rotate on r parameters
parameter['use_rotate_on_r'] = True
parameter['rotate_on_r_w1'] = 0.4
parameter['rotate_on_r_w2'] = 0.2
parameter['rotate_on_r_w3'] = 0.1
parameter['rotate_on_r_quad_coef'] = 0.000006
parameter['rotate_on_r_max_iter'] = 200
parameter['rotate_on_r_eps'] = 0.0035
parameter['rotate_on_r_step_eps'] = 0.0000

# rotate on theta parameters
parameter['use_rotate_on_theta'] = False
parameter['rotate_on_theta_eps'] = 0.0028
parameter['rotate_on_theta_max_iter'] = 720

# shift on z parameters
parameter['use_shift_on_z'] = True

# multiprocess
parameter['use_multiprocess'] = True

####
clusterer = Cluster(data.loc[data.track_id.isnull()], parameter)
result = clusterer.run()
####

# create track pool

parameter={}
parameter['track_length_1_min']=10
parameter['track_length_1_max']=20
parameter['use_multiprocess'] = True
parameter['is_training'] = False

####
pool = TrackPool(data.loc[data.track_id.isnull()], result, parameter)
print(pool.length_1)
print(pool.length_2)
####

# merge_1
parameter={}
parameter['is_training'] = False
parameter['track_length_2_min'] = 10
parameter['track_length_2_max'] = 20
parameter['track_length_diff_max'] = 5

####
output = Merge_1(pool, parameter)
print(len(output))
temp = data.merge(output, how='left', on='hit_id', suffixes=('_old', ''))
data.update(temp.track_id)
####
data.head()

# score
print(Score(event_id, data, truth))
print(len(data.loc[data.track_id.notnull()]))
print(len(data.loc[data.track_id.notnull()])/len(data))
score.append((parameter['track_length_2_min'], parameter['track_length_diff_max'], Score(event_id, data, truth)))

100%|██████████| 200/200 [03:09<00:00,  1.05it/s]


46259
5020
15014
0.4766338812877876
52973
0.4380142055085622
CPU times: user 1min 22s, sys: 10.6 s, total: 1min 33s
Wall time: 4min 58s


In [8]:
%%time
# track with length>=10
# cluster

parameter={}

# rotate on r parameters
parameter['use_rotate_on_r'] = False
parameter['rotate_on_r_w1'] = 0.4
parameter['rotate_on_r_w2'] = 0.2
parameter['rotate_on_r_w3'] = 0.1
parameter['rotate_on_r_quad_coef'] = 0.000006
parameter['rotate_on_r_max_iter'] = 200
parameter['rotate_on_r_eps'] = 0.0035
parameter['rotate_on_r_step_eps'] = 0.0000

# rotate on theta parameters
parameter['use_rotate_on_theta'] = True
parameter['rotate_on_theta_eps'] = 0.0028
parameter['rotate_on_theta_max_iter'] = 1800

# shift on z parameters
parameter['use_shift_on_z'] = True

# multiprocess
parameter['use_multiprocess'] = True

####
clusterer = Cluster(data.loc[data.track_id.isnull()], parameter)
result = clusterer.run()
####

# create track pool

parameter={}
parameter['track_length_1_min']=10
parameter['track_length_1_max']=20
parameter['use_multiprocess'] = True
parameter['is_training'] = False

####
pool = TrackPool(data.loc[data.track_id.isnull()], result, parameter)
print(pool.length_1)
print(pool.length_2)
####

# merge_1
parameter={}
parameter['is_training'] = False
parameter['track_length_2_min'] = 10
parameter['track_length_2_max'] = 20
parameter['track_length_diff_max'] = 5

####
output = Merge_1(pool, parameter)
print(len(output))
temp = data.merge(output, how='left', on='hit_id', suffixes=('_old', ''))
data.update(temp.track_id)
####
data.head()

# score
print(Score(event_id, data, truth))
print(len(data.loc[data.track_id.notnull()]))
print(len(data.loc[data.track_id.notnull()])/len(data))
score.append((parameter['track_length_2_min'], parameter['track_length_diff_max'], Score(event_id, data, truth)))

100%|██████████| 1800/1800 [14:41<00:00,  2.04it/s]


12642
4947
8646
0.5308249010528386
61619
0.5095047916718346
CPU times: user 1min 30s, sys: 22.1 s, total: 1min 52s
Wall time: 16min 33s


In [9]:
%%time
# track with length>=8
# cluster

parameter={}

# rotate on r parameters
parameter['use_rotate_on_r'] = True
parameter['rotate_on_r_w1'] = 0.4
parameter['rotate_on_r_w2'] = 0.2
parameter['rotate_on_r_w3'] = 0.1
parameter['rotate_on_r_quad_coef'] = 0.000006
parameter['rotate_on_r_max_iter'] = 200
parameter['rotate_on_r_eps'] = 0.0037
parameter['rotate_on_r_step_eps'] = 0.0000

# rotate on theta parameters
parameter['use_rotate_on_theta'] = False
parameter['rotate_on_theta_eps'] = 0.0028
parameter['rotate_on_theta_max_iter'] = 720

# shift on z parameters
parameter['use_shift_on_z'] = True

# multiprocess
parameter['use_multiprocess'] = True

clusterer = Cluster(data.loc[data.track_id.isnull()], parameter)
result = clusterer.run()


# create track pool
parameter={}
parameter['track_length_1_min']=8
parameter['track_length_1_max']=20
parameter['use_multiprocess'] = True
parameter['is_training'] = False

pool = TrackPool(data.loc[data.track_id.isnull()], result, parameter)
print(pool.length_1)
print(pool.length_2)


# merge_1
parameter={}
parameter['is_training'] = False
parameter['track_length_2_min'] = 8
parameter['track_length_2_max'] = 20
parameter['track_length_diff_max'] = 5

output = Merge_1(pool, parameter)
print(len(output))
temp = data.merge(output, how='left', on='hit_id', suffixes=('_old', ''))
data.update(temp.track_id)

# score
print(Score(event_id, data, truth))
print(len(data.loc[data.track_id.notnull()]))
print(len(data.loc[data.track_id.notnull()])/len(data))
score.append((parameter['track_length_2_min'], parameter['track_length_diff_max'], Score(event_id, data, truth)))

100%|██████████| 200/200 [02:13<00:00,  1.49it/s]


14638
3015
7804
0.5926145246943046
69423
0.574033190286012
CPU times: user 46 s, sys: 3.79 s, total: 49.8 s
Wall time: 3min 8s


In [10]:
%%time
# track with length>=8
# cluster

parameter={}

# rotate on r parameters
parameter['use_rotate_on_r'] = False
parameter['rotate_on_r_w1'] = 0.4
parameter['rotate_on_r_w2'] = 0.2
parameter['rotate_on_r_w3'] = 0.1
parameter['rotate_on_r_quad_coef'] = 0.000006
parameter['rotate_on_r_max_iter'] = 200
parameter['rotate_on_r_eps'] = 0.0037
parameter['rotate_on_r_step_eps'] = 0.0000

# rotate on theta parameters
parameter['use_rotate_on_theta'] = True
parameter['rotate_on_theta_eps'] = 0.0028
parameter['rotate_on_theta_max_iter'] = 1800

# shift on z parameters
parameter['use_shift_on_z'] = True

# multiprocess
parameter['use_multiprocess'] = True

clusterer = Cluster(data.loc[data.track_id.isnull()], parameter)
result = clusterer.run()


# create track pool
parameter={}
parameter['track_length_1_min']=8
parameter['track_length_1_max']=20
parameter['use_multiprocess'] = True
parameter['is_training'] = False

pool = TrackPool(data.loc[data.track_id.isnull()], result, parameter)
print(pool.length_1)
print(pool.length_2)


# merge_1
parameter={}
parameter['is_training'] = False
parameter['track_length_2_min'] = 8
parameter['track_length_2_max'] = 20
parameter['track_length_diff_max'] = 5

output = Merge_1(pool, parameter)
print(len(output))
temp = data.merge(output, how='left', on='hit_id', suffixes=('_old', ''))
data.update(temp.track_id)

# score
print(Score(event_id, data, truth))
print(len(data.loc[data.track_id.notnull()]))
print(len(data.loc[data.track_id.notnull()])/len(data))
score.append((parameter['track_length_2_min'], parameter['track_length_diff_max'], Score(event_id, data, truth)))

100%|██████████| 1800/1800 [09:52<00:00,  3.04it/s]


10251
3201
5068
0.614727259824818
74491
0.6159386136812773
CPU times: user 1min 7s, sys: 17.4 s, total: 1min 24s
Wall time: 11min 20s


In [11]:
%%time
# track with length>=6
# cluster

parameter={}

# rotate on r parameters
parameter['use_rotate_on_r'] = True
parameter['rotate_on_r_w1'] = 0.4
parameter['rotate_on_r_w2'] = 0.2
parameter['rotate_on_r_w3'] = 0.1
parameter['rotate_on_r_quad_coef'] = 0.000006
parameter['rotate_on_r_max_iter'] = 200
parameter['rotate_on_r_eps'] = 0.0037
parameter['rotate_on_r_step_eps'] = 0.0000

# rotate on theta parameters
parameter['use_rotate_on_theta'] = False
parameter['rotate_on_theta_eps'] = 0.0028
parameter['rotate_on_theta_max_iter'] = 720

# shift on z parameters
parameter['use_shift_on_z'] = True

# multiprocess
parameter['use_multiprocess'] = True

clusterer = Cluster(data.loc[data.track_id.isnull()], parameter)
result = clusterer.run()


# create track pool
parameter={}
parameter['track_length_1_min']=6
parameter['track_length_1_max']=20
parameter['use_multiprocess'] = True
parameter['is_training'] = False

pool = TrackPool(data.loc[data.track_id.isnull()], result, parameter)
print(pool.length_1)
print(pool.length_2)


# merge_1
parameter={}
parameter['is_training'] = False
parameter['track_length_2_min'] = 6
parameter['track_length_2_max'] = 20
parameter['track_length_diff_max'] = 6

output = Merge_1(pool, parameter)
print(len(output))
temp = data.merge(output, how='left', on='hit_id', suffixes=('_old', ''))
data.update(temp.track_id)

# score
print(Score(event_id, data, truth))
print(len(data.loc[data.track_id.notnull()]))
print(len(data.loc[data.track_id.notnull()])/len(data))
score.append((parameter['track_length_2_min'], parameter['track_length_diff_max'], Score(event_id, data, truth)))

100%|██████████| 200/200 [01:42<00:00,  1.96it/s]


13537
3314
7202
0.6428050700116028
81693
0.6754892962567907
CPU times: user 46.5 s, sys: 3.28 s, total: 49.7 s
Wall time: 2min 36s


In [12]:
%%time
# everything else
# cluster

parameter={}

# rotate on r parameters
parameter['use_rotate_on_r'] = False
parameter['rotate_on_r_w1'] = 0.4
parameter['rotate_on_r_w2'] = 0.2
parameter['rotate_on_r_w3'] = 0.1
parameter['rotate_on_r_quad_coef'] = 0.000006
parameter['rotate_on_r_max_iter'] = 200
parameter['rotate_on_r_eps'] = 0.0037
parameter['rotate_on_r_step_eps'] = 0.0000

# rotate on theta parameters
parameter['use_rotate_on_theta'] = True
parameter['rotate_on_theta_eps'] = 0.0029
parameter['rotate_on_theta_max_iter'] = 720

# shift on z parameters
parameter['use_shift_on_z'] = True

# multiprocess
parameter['use_multiprocess'] = True

clusterer = Cluster(data.loc[data.track_id.isnull()], parameter)
result = clusterer.run()

# create track pool
parameter={}
parameter['track_length_1_min']=4
parameter['track_length_1_max']=20
parameter['use_multiprocess'] = True
parameter['is_training'] = False

pool = TrackPool(data.loc[data.track_id.isnull()], result, parameter)
print(pool.length_1)
print(pool.length_2)

# merge_1
parameter={}
parameter['is_training'] = False
parameter['track_length_2_min'] = 4
parameter['track_length_2_max'] = 20
parameter['track_length_diff_max'] = 8

output = Merge_1(pool, parameter)
print(len(output))
temp = data.merge(output, how='left', on='hit_id', suffixes=('_old', ''))
data.update(temp.track_id)

# score
print(Score(event_id, data, truth))
print(len(data.loc[data.track_id.notnull()]))
print(len(data.loc[data.track_id.notnull()])/len(data))
score.append((parameter['track_length_2_min'], parameter['track_length_diff_max'], Score(event_id, data, truth)))

100%|██████████| 720/720 [02:56<00:00,  4.07it/s]


193163
44395
29266
0.6518725719091625
110959
0.9174790596912493
CPU times: user 8min 58s, sys: 10.5 s, total: 9min 8s
Wall time: 13min 5s


In [13]:
# fill na

output = Merge_2(data)
print(len(output))
temp = data.merge(output, how='left', on='hit_id', suffixes=('_old', ''))
data.update(temp.track_id)

assert len(data.loc[data.track_id.isnull()]) == 0

# convert track_id to integer
data.track_id = data.track_id.astype('int')

9980


In [14]:
# extend
for i in range(0, 4):
    data = Extend(data)
    print(Score(event_id, data, truth))
    score.append((-1,-1,Score(event_id, data, truth)))

0.6622355705483373
0.6627120628238552
0.6619702271169725
0.663535632779459


In [15]:
score

[(14, 4, 0.09350334346239389),
 (12, 4, 0.2855895408751131),
 (12, 4, 0.32533472275248315),
 (10, 5, 0.4766338812877876),
 (10, 5, 0.5308249010528386),
 (8, 5, 0.5926145246943046),
 (8, 5, 0.614727259824818),
 (6, 6, 0.6428050700116028),
 (4, 8, 0.6518725719091625),
 (-1, -1, 0.6622355705483373),
 (-1, -1, 0.6627120628238552),
 (-1, -1, 0.6619702271169725),
 (-1, -1, 0.663535632779459)]